<a href="https://colab.research.google.com/github/khalil649/Big-Data/blob/main/notebooks/05.00-Big-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big data


Loading dataset

In [2]:
!pip install pymongo
import subprocess
# Start MongoDB service (requires sudo privileges)

!systemctl start mongod

import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['movielens']
movies_collection = db['movies']

# Load movies
movies_df = pd.read_csv('data/movies.csv')
movies_df['genres'] = movies_df['genres'].str.split('|')

# Merge with ratings
ratings_df = pd.read_csv('data/ratings.csv')
ratings_df['rating'] = ratings_df['rating'].astype(float)

# Group ratings by movieId
ratings_summary = ratings_df.groupby('movieId').agg(
    avg_rating=('rating', 'mean'),
    total_ratings=('rating', 'count'),
    all_ratings=('rating', list)
).reset_index()

# Merge movies with ratings
movies_with_ratings = pd.merge(movies_df, ratings_summary, on='movieId', how='left')

# Convert DataFrame to list of dicts
movies_list = movies_with_ratings.to_dict(orient='records')

# Insert into MongoDB
movies_collection.delete_many({})
movies_collection.insert_many(movies_list)

print("✅ Data successfully loaded into MongoDB")

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 685805267c92a0ec9a8661cb, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>